# **Bringing AI to the Edge!**
In an era where data privacy is paramount, Federated Learning (FL) and Retrieval-Augmented Generation (RAG) offer a groundbreaking approach to building AI models. By training AI across decentralized devices without sharing sensitive data, FL ensures privacy while continuously improving AI capabilities. With RAG, we unlock the potential of enhanced data retrieval for smarter, more efficient models.



In [1]:
!pip install --quiet tensorflow_federated

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 16.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.4/58.4 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 31.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 MB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.8/104.8 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 721.6/721.6 kB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.9/230.9 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.7/73.7 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 37.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import tensorflow_federated as tff
import tensorflow as tf
def create_model():
  model = tf.keras.models.Sequential([
      tf.keras.layers.Dense(10, activation='relu', input_shape=(28,28)),
      tf.keras.layers.Dense(10, activation='softmax'),
  ])
  return model



/usr/local/lib/python3.10/dist-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
ERROR:jax._src.xla_bridge:Jax plugin configuration error: Exception when calling jax_plugins.xla_cuda12.initialize()
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/jax/_src/xla_bridge.py", line 438, in discover_pjrt_plugins
    plugin_module.initialize()
  File "/usr/local/lib/python3.10/dist-packages/jax_plugins/xla_cuda12/__init__.py", line 85, in initialize
    options = xla_client.generate_pjrt_gpu_plugin_options()
AttributeError: module 'jaxlib.xla_client' has no attribute 'generate_pjrt_gpu_plugin_options'


Data distribution accross client

In [3]:
from google.colab import files
uploaded=files.upload()

import pandas as pd
client_data=pd.read_csv('Mall_Customers.csv')
client_data


Saving Mall_Customers.csv to Mall_Customers.csv


,CustomerID,Genre,Age,Annual_Income_(k$),Spending_Score
0,1,Male,19,15,39
1,2,Male,21,15,81
2,3,Female,20,16,6
3,4,Female,23,16,77
4,5,Female,31,17,40
...,...,...,...,...,...
195,196,Female,35,120,79
196,197,Female,45,126,28
197,198,Male,32,126,74
198,199,Male,32,137,18


In [4]:
def process_data(data):
  return tf.data.Dataset.from_tensor_slices(data).batch(32)

Processing data for each client

Data : A Pandas DataFrame containing the input data.
    
Return : A TensorFlow Dataset.

In [7]:
import tensorflow as tf
import pandas as pd

def process_data(data):
  feature_dict = {col: data[col].tolist() for col in data.columns}
  return tf.data.Dataset.from_tensor_slices(feature_dict).batch(32)
federated_train_data = [
    process_data(client_data)
]



Debugging deep learning: Passing input_ids as a keyword argument to DPRQuestionEncoder fixed the 'TypeError: argument after ** must be a mapping, not Tensor' error. #python #transformers #huggingface

In [10]:
from transformers import DPRQuestionEncoder , DPRQuestionEncoderTokenizer
tokenizer = DPRQuestionEncoderTokenizer.from_pretrained('facebook/dpr-question_encoder-single-nq-base')
model = DPRQuestionEncoder.from_pretrained('facebook/dpr-question_encoder-single-nq-base')
input_ids = tokenizer("'what is federated learning", return_tensors="pt")["input_ids"]
outputs = model(input_ids=input_ids)



Some weights of the model checkpoint at facebook/dpr-question_encoder-single-nq-base were not used when initializing DPRQuestionEncoder: ['question_encoder.bert_model.pooler.dense.bias', 'question_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRQuestionEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRQuestionEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


preparing data and a model for federated learning.

processing a Pandas DataFrame, separating features and labels for training.

defining a simple neural network model using Keras and converts it into a format suitable for TensorFlow Federated.

This sets the stage for training the model across decentralized datasets.

In [12]:
!pip install --quiet --upgrade tensorflow-federated

In [19]:
import tensorflow as tf
import pandas as pd
def process_data(data):
  features = data.drop(columns=['Spending_Score'])
  labels = data['Spending_Score']
  dataset = tf.data.Dataset.from_tensor_slices((features.to_dict('list'), labels.tolist()))
  return dataset.batch(32)

federated_train_data = [
    process_data(client_data)
]

def create_model():
  keras_model = tf.keras.models.Sequential([
      tf.keras.layers.Dense(10, activation='relu', input_shape=(4,)),
      tf.keras.layers.Dense(10, activation='softmax'),
  ])
  element_spec = federated_train_data[0].element_spec
  return tff.learning.models.from_keras_model(
      keras_model,
      input_spec=element_spec,
      loss=tf.keras.losses.CategoricalCrossentropy()
  )

Quantizing a PyTorch model for efficiency. Make sure to initialize your 'retrieval_model' first!

In [22]:
import torch
retrieval_model = torch.nn.Linear(10, 10)

model_fp32 = retrieval_model
model_int8 = torch.quantization.quantize_dynamic(
    model_fp32, {torch.nn.Linear}, dtype=torch.qint8
)
model_fp32 = model_int8.float()

Evaluating a classification model using precision and recall metrics in sklearn. This code compares true labels (y_true) with predicted labels (y_pred) using macro averaging to provide an overall performance assessment. Precision indicates the accuracy of positive predictions, while recall measures the ability to identify all positive cases.

In [31]:
from sklearn.metrics import precision_score, recall_score
y_true = [0, 1, 2, 0, 1, 2]
y_pred = [0, 2, 1, 0, 0, 1]
precision = precision_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')

print(f"Precision: {precision}")
print(f"Recall: {recall}")

Precision: 0.2222222222222222
Recall: 0.3333333333333333


Federated Learning and Retrieval-Augmented Generation empower developers to create intelligent, privacy-first AI systems. With a focus on decentralization, data security, and advanced retrieval techniques, this paradigm reshapes how we approach AI training and deployment.

Whether you're exploring new architectures or implementing robust privacy mechanisms, the concepts in this notebook equip you with the tools to lead the way in the next wave of AI innovation.